In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_openai langchain_community langchain_core 

In [2]:
import os, getpass
from langchain_openai import ChatOpenAI
from IPython.display import Image, display
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, RemoveMessage
from langgraph.graph import MessagesState
import sqlite3
from typing import Optional, Literal, List
from pydantic import BaseModel, Field
from langchain_core.runnables import RunnableConfig
from langgraph.store.base import BaseStore

In [3]:
## Set environment variables and API KEYS
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "coach_draft"

In [4]:
# Create my more llm model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0) 

## Create in-chat and cross-chats memory


In [5]:
# In memory DB
conn = sqlite3.connect(":memory:", check_same_thread = False)

# TODO: Connect to an external or in memory DB

# Here is our checkpointer 
from langgraph.checkpoint.sqlite import SqliteSaver
checkpoint_memory = SqliteSaver(conn)

# Long term memory
from langgraph.store.memory import InMemoryStore
longterm_memory = InMemoryStore()

## Prompts repo

In [6]:
# TODO: maybe separate the overall prompt into a personality prompt and more granular ones per task.
overall_system_prompt = """
You are a professional development and growth coach. Your role is to guide the user through self-reflection, goal setting, skill improvement, and career planning. Your responses should be empathetic, insightful, and tailored to the user's unique context. Ask clarifying questions when needed, and help the user explore practical strategies, habits, or frameworks to make meaningful progress.
    
Behave like a mix of a mentor and coach—motivating, structured, and supportive. Provide suggestions backed by reasoning, using examples when helpful. Use a conversational but professional tone. When discussing plans or actions, guide the user to break them down into small, achievable steps. Occasionally summarize the user's progress and reflect it back to them to encourage continuity.

You will be having realtime conversations, thus avoid walls of text, and use instead an interactive approach with shorter statements, few sentences per turn, stimulating dialog rather than monologues. e.g. avoid outputting long bulleted lists except when really unavoidable (e.g. you can list agreed steps, but don't write list after list while discussing the steps).

In instructions below you will receive specific information about the patient, the ongoing conversation, and your goals in this part of the conversation. While preserving the personality above, focus on getting the job described below done.
"""
# This is to limit the sessions lenght
# TODO: ideally combine turns with time as a stop condition
n_turns_onboarding = 4 # Can be small for testing
onboarding_promt = f"""
    In this phase of the conversation your goal is to define an overall plan for the coaching program. 
    You will discuss with the user what are their goals, what's your suggested structure for the program, and agree on how it will be ran. 
    Stick to this structure:
    
    1. Onboarding (this session): get to know the user, discuss goals and expectations.
    2. Goal Setting (also in this session): define 2–3 clear, achievable goals.
    3. Weekly Sessions: propose 1 or 2 sessions/week of 30 to 60 minutes for 4–6 weeks to make progress.
    4. Always-available chat: explain that the user can reach out anytime for questions or exercises.
    5. Closing: after 4–6 weeks, review progress and define next steps.
    
    Check with the user if they want to adjust anything. Keep it simple and focused.
    This phase chat should last for about {n_turns_onboarding} turns, when approaching that number make sure to start wrapping up. This is for debugging, so sessions may appear a bit short.
    Once you are done with the session and there is no follow up questions for the user, thank the user and say good bye. 
    The session will finish by this point, so don't leave anything hanging before thanking and saying good bye.
    """

# This is to limit the sessions lenght
# TODO: ideally combine turns with time as a stop condition
n_turns_session = 5 # Can be small for testing
session_prompt = f"""
In this phase of the conversation, your goal is to conduct a regular coaching session as part of the ongoing program.

You have access to the following information:
- The overall coaching plan.
- A summary of past sessions.
- Any homework assigned from the last session.

Use this information to inform the current session.

Stick to this structure:

1. **Check-in**: Greet the user and ask how they've been since the last session. Discuss any updates or changes in their circumstances.
2. **Session Planning**: Explain and agree on a plan for this session. Set specific goals for the session, such as reviewing past agreements, updating them, and developing a plan for the upcoming days.
3. **Review**: Discuss the progress made on the goals set previously. Reflect on the homework assigned and any challenges faced.
4. **Guidance**: Utilize coaching techniques that encourage the user to define their own actions. Ask open-ended questions to facilitate self-discovery. Provide tools or guidance if needed, but prioritize empowering the user to develop their own solutions.
5. **Plan**: Collaboratively outline the focus areas and action items for the upcoming week.
6. **Wrap-up**: Summarize the session, confirm the next meeting schedule, and thank the user for their participation.

Ensure the session is interactive, allowing the user to share their experiences and thoughts. This phase chat should last for about {n_turns_session} turns. Once you are done with the session, thank the user and say goodbye.
"""

## Classes

In [7]:
class State(MessagesState):
    """Overall state to keep track an ongoing conversation"""
    summary: str
    turn_count: int = Field(description="Which converstaion turn are we at.", default=0)
    finished: bool = Field(
        default=False,
        description="Whether the discussion is finished"
    )
    # TODO: if we want to have more granular control of the session, can use a this to plan and track a session progress
    # current_session : Optional[Session] = Field(
    #     description="Parameters of the session currently in progress", 
    #     default_factory=Session)

class Session(BaseModel):
    """This is the overall plan for one coaching session with the user"""
    session_summary: Optional[str] = Field(description="The overall summary of this session, what was done", default=None)
    homework: List[str] = Field(description="List of tasks agreed with the user for the next session", default_factory=list)
    ## TODO: The elements below can be added to give more structure to the session, instead of creating the structure in the prompt.
    # plan_desciption: Optional[str] = Field(description="The overall plan description for this session, what is intended to do", default=None)
    # session_goals: List[str] = Field(description="List of goals agreed with the user to work on this current session", default_factory=list)
    # phase: Literal['not_started','kickstart', 'discussion', 'clossing', 'closed'] = Field(
    #     default='not_started',
    #     description="Current phase of the session plan"
    # )
    
class CoachingPlan(BaseModel):
    """This is the overall plan for coaching the user"""
    plan_desciption: Optional[str] = Field(
        description="The overall plan description, what it intends to achieve and how we intend to achieve it accross several sessions.", 
        default=None)
    plan_summary: Optional[str] = Field(
        description="Summary of the plan up to this point. It should give an overview that will help define next steps", 
        default=None)
    past_sessions: List[Session] = Field(description="List of sessions held so far", default_factory=list)
    goals: List[str] = Field(description="List of goals agreed with the user", default_factory=list)
    specific_instructions: Optional[str] = Field(
        description=(
            "Any specific instructions for the overall coaching plan agreed with the user"
            "e.g. which days to hold the sessions, how to address the user, what aspects to focus on, etc"
        ), 
        default=None)
    ## TODO: next sessions and phases can be implemented later to give more structure to the plan
    # current_phase: Literal['onboarding','sessions','clossing'] = Field(
    #         default='onboarding',
    #         description="Current phase of the coaching plan: onboarding is the first session, then comes the followup sessions, and then the closing one"
    #     )
    # next_sessions: List[Session] = Field(description="List of sessions to be had", default_factory=list)

## Common functions

In [19]:
message_history_size = 3
def summarize_conversation(state: State):
    """Creates a summary of the ongoing conversation and deletes all but the last messages"""
    # First, we get any existing summary
    summary = state.get("summary", "")

    summary_message = (
        f"This is summary of the conversation to date: {summary}\n\n"
        "Extend the summary (or start it if empty) by taking into account the new messages in the conversation."
        "Note that old messages will be deleted, thus make sure to include important facts, while keeping it concise."
    )

    # Add prompt to our history
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = llm.invoke(messages)
    
    # Delete all but the N most recent messages
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-message_history_size]]
    return {"summary": response.content, "messages": delete_messages}

class FinishDecision(BaseModel):
    """To be use to structure the LLM output when deciding to stop or continue a conversation"""
    decision: Literal["finish", "continue"]


def check_if_finished(state: State):
    """
        Decide if the session is finished. 
    """
    system_message = """
        Below is a conversation between a coach and a coachee.
        Determine if the conversation is finished or still ongoing based on whether the coachee has closed it
        e.g. thanks the user, say good bye, see you later, etc
        Make sure there are no pending questions or call to actions for the user, since after deciding to finish the session will close and the user won't have the chance to reply.
        If there are any pending questions or call to actions for the user, ALWAYS continue the conversation. ONLY when the coach has stopped asking follow ups to the user, decide to finish the session.
        """

    # Get summary if it exists
    summary = state.get("summary", "No summary yet")
    system_message += f"\n Summary of conversation earlier:\n{summary}"
    
    messages = [SystemMessage(system_message)] + state["messages"]
    decision = llm.with_structured_output(FinishDecision).invoke(messages)

    if decision.decision == "continue":
        # We are not finished with onboarding. Summarize if needed, trim messages and continue the conversation
        messages = state["messages"]
        # If there are more than X messages, then we summarize the conversation
        if len(messages) > message_history_size:
            return "summarize_conversation"
        else:
            return END
    else:
        # the session is finished, we need to summarize the session, update the coaching plan, and finish
        return "finalize_session"

def summarize_session(state: State) -> Session:
    """
        Summarize this session in the format of a Session class.
    """
    system_message = """
        Below is a conversation between a coach and a coachee.
        Summarize this session in the format of a Session. Fill in all the fields.
        Add as homework any tasks planned or foreseen for the next session.
        """

    # Get summary if it exists
    summary = state.get("summary", "No summary yet")
    system_message += f"\n Summary of conversation earlier:\n{summary}"
    
    messages = [SystemMessage(system_message)] + state["messages"]
    system_prompt = "Summarize this session in the format of a Session."
    messages = [SystemMessage(system_prompt)] + state["messages"]
    session_summary = llm.with_structured_output(Session).invoke(messages)
    return session_summary

def finalize_session(state: State, config: RunnableConfig, store: BaseStore) -> dict:
    user_id = config["configurable"]["user_id"]
    namespace = ("session_plan", user_id)
    memories = store.search(namespace)

    # Retrieve existing CoachingPlan or initialize a new one
    if memories:
        coaching_plan = CoachingPlan(**memories[0].value)
    else:
        coaching_plan = CoachingPlan()

    # Update the past sessions independently to prevent the LLM overwriting past sessions accidentally
    past_sessions = coaching_plan.past_sessions + [summarize_session(state)]
    
    # Prepare the system prompt for the LLM
    system_message = f"""
    You are a coaching assistant. Based on the recent conversation, update the coaching plan accordingly.
    - Do not modify the 'sessions' field; it will be updated separately.
    - Update 'goals' and 'specific_instructions' if new information is present.
    - If 'plan_desciption' is not set, provide a concise summary of the overall plan. If new information during this conversation makes it relevant, update the overall plan.
    - Update the plan_summary adding any new relevant information from this session, while preserving enough details about the past sessions to understand how the plan has progressed up to this point.
    Existing CoachingPlan:
    {coaching_plan.model_dump_json(indent=2)}
    """

    # Get summary if it exists
    summary = state.get("summary", "No summary yet")
    system_message += f"\n Summary of conversation earlier:\n{summary}"
    
    # Combine the system prompt with the conversation messages
    messages = [SystemMessage(system_message)] + state["messages"]

    # Invoke the LLM to get the updated CoachingPlan
    updated_plan = llm.with_structured_output(CoachingPlan).invoke(messages)

    # Fill in the past sessions list
    updated_plan.past_sessions = past_sessions

    # Store the updated CoachingPlan
    store.put(namespace, "current_plan", updated_plan.model_dump())
    
    return {'finished':True}

# Onboarding subgraph
Given that onboarding is a one-off per coaching plan, I rather have it as a separate graph. 
It could be added to the ongoing graph as a conditional edge to allow for replanning.

In [10]:
def call_model_onboarding(state: State, config: RunnableConfig, store: BaseStore):
        
    # Initialise system message
    system_message = overall_system_prompt

    # Add section specific instructions
    system_message += onboarding_promt

    # Get turn
    turn_count = state.get("turn_count", 0)
    system_message += f"\nYou are now at this turn number in the conversation: {turn_count}"
    
    # Get summary if it exists
    summary = state.get("summary", "No summary yet")
    system_message += f"\n Summary of conversation earlier:\n{summary}"  
    
    system_message += "\n\nLast messages:\n"
    
    messages = [SystemMessage(system_message)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": response, "turn_count": turn_count+1}

In [11]:
# Define a new graph
workflow = StateGraph(State)
workflow.add_node("conversation", call_model_onboarding)
workflow.add_node(summarize_conversation)
workflow.add_node(finalize_session)

# Set the entrypoint as conversation
workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", check_if_finished)
workflow.add_edge("summarize_conversation", END)
workflow.add_edge("finalize_session", END)

# Compile
graph = workflow.compile(checkpointer=checkpoint_memory, store=longterm_memory)
# display(Image(graph.get_graph().draw_mermaid_png())) --> the server isn't working 
ascii_diagram = graph.get_graph().draw_ascii()
print(ascii_diagram)

                                    +-----------+                            
                                    | __start__ |                            
                                    +-----------+                            
                                           *                                 
                                           *                                 
                                           *                                 
                                   +--------------+                          
                                   | conversation |                          
                              .....+--------------+......                    
                         .....             .             ......              
                   ......                   .                  .....         
                ...                         .                       ......   
+------------------+           +------------------------+       

In [12]:
# Create a thread
import uuid
thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id, "user_id": "Pablo"}}

In [13]:
for i in range(10):
    input_message = HumanMessage(content=input())
    output = graph.invoke({"messages": [input_message]}, config) 
    for m in output['messages']:
        m.pretty_print()
    if 'summary' in output.keys():
        print('\n----------\nSummary:\n',output['summary'])
    if checkpoint_memory.get(config)['channel_values'].get("finished")==True:
        print('CONVERSATION FINISHED')
        break
# Handy content:  hi! I'm Pablo, I'm looking for a new job. I want to bring my linkedin to life, improve my dutch, and network more actively. I want to reach C1 level of dutch, connect to 1 or 2 new people every day, and post often. For dutch, I want to follow a course which I found, and join a weekly chat club. For networking I want to do it in linkedin, and for posting I want to focus on genAI latest advancements

 hi! I'm Pablo, I'm looking for a new job. I want to bring my linkedin to life, improve my dutch, and network more actively. I want to reach C1 level of dutch, connect to 1 or 2 new people every day, and post often. For dutch, I want to follow a course which I found, and join a weekly chat club. For networking I want to do it in linkedin, and for posting I want to focus on genAI latest advancements


================================ Human Message =================================

hi! I'm Pablo, I'm looking for a new job. I want to bring my linkedin to life, improve my dutch, and network more actively. I want to reach C1 level of dutch, connect to 1 or 2 new people every day, and post often. For dutch, I want to follow a course which I found, and join a weekly chat club. For networking I want to do it in linkedin, and for posting I want to focus on genAI latest advancements
================================== Ai Message ==================================

Hi Pablo! It’s great to meet you. You have some clear and ambitious goals, which is fantastic! Let’s break this down a bit to create a structured plan that will help you achieve them.

First, can you tell me a bit more about your current level of Dutch and how comfortable you feel with networking on LinkedIn? This will help us tailor the plan to your needs.


 I'm level B1 in dutch, and comfortable but lazy in linkedin


I'm asking to summarize
================================== Ai Message ==================================

Hi Pablo! It’s great to meet you. You have some clear and ambitious goals, which is fantastic! Let’s break this down a bit to create a structured plan that will help you achieve them.

First, can you tell me a bit more about your current level of Dutch and how comfortable you feel with networking on LinkedIn? This will help us tailor the plan to your needs.
================================ Human Message =================================

I'm level B1 in dutch, and comfortable but lazy in linkedin
================================== Ai Message ==================================

Thanks for sharing that! It sounds like you have a solid foundation in Dutch and a good starting point for networking, but we can definitely work on building your motivation and consistency.

Here’s a proposed structure for our coaching program:

1. **Onboarding**: This session, where we’re getting to know ea

 no, is ok


I'm asking to summarize
================================== Ai Message ==================================

Thanks for sharing that! It sounds like you have a solid foundation in Dutch and a good starting point for networking, but we can definitely work on building your motivation and consistency.

Here’s a proposed structure for our coaching program:

1. **Onboarding**: This session, where we’re getting to know each other and discussing your goals.
2. **Goal Setting**: We’ll define 2-3 clear, achievable goals based on what you’ve shared.
3. **Weekly Sessions**: I suggest we meet 1-2 times a week for 30 to 60 minutes over the next 4-6 weeks to track your progress and adjust as needed.
4. **Always-Available Chat**: You can reach out anytime with questions or for exercises.
5. **Closing**: After 4-6 weeks, we’ll review your progress and define the next steps.

How does that sound to you? Would you like to adjust anything?
================================ Human Message =====================

 sounds good. I have to go, can we close the call now?


================================== Ai Message ==================================

Thanks for sharing that! It sounds like you have a solid foundation in Dutch and a good starting point for networking, but we can definitely work on building your motivation and consistency.

Here’s a proposed structure for our coaching program:

1. **Onboarding**: This session, where we’re getting to know each other and discussing your goals.
2. **Goal Setting**: We’ll define 2-3 clear, achievable goals based on what you’ve shared.
3. **Weekly Sessions**: I suggest we meet 1-2 times a week for 30 to 60 minutes over the next 4-6 weeks to track your progress and adjust as needed.
4. **Always-Available Chat**: You can reach out anytime with questions or for exercises.
5. **Closing**: After 4-6 weeks, we’ll review your progress and define the next steps.

How does that sound to you? Would you like to adjust anything?
================================ Human Message =================================

no, is ok


In [14]:
namespace = ("session_plan", 'Pablo')
memories = longterm_memory.search(namespace)

coaching_plan = CoachingPlan(**memories[0].value)
print(coaching_plan.model_dump_json(indent=2))

{
  "plan_desciption": "This coaching plan aims to help Pablo enhance his professional profile and language skills, specifically targeting a C1 level in Dutch and improving his LinkedIn networking efforts.",
  "plan_summary": "Pablo is currently at a B1 level in Dutch and aims to reach C1 by following a course and joining a weekly chat club. He also wants to actively network on LinkedIn by connecting with 1-2 new people daily and posting about generative AI. The coaching program includes onboarding, goal setting, weekly sessions for progress tracking, an always-available chat for questions or exercises, and a closing session to review progress and define next steps.",
  "past_sessions": [
    {
      "session_summary": "In this session, we established a coaching program focused on achieving C1 level in Dutch and enhancing LinkedIn networking. We discussed meeting 1-2 times a week for the next 4-6 weeks and maintaining open communication for questions and exercises.",
      "homework": 

# Sessions bot

In [15]:
def call_model_session(state: State, config: RunnableConfig, store: BaseStore):
    # Initialise system message
    system_message = overall_system_prompt

    # Add section specific instructions
    system_message += session_prompt

    # Get the user ID from the config
    user_id = config["configurable"]["user_id"]

    namespace = ("session_plan", user_id)
    memories = longterm_memory.search(namespace)
    # should add a try catch or a default here in case there is no coaching plan in memory
    coaching_plan = CoachingPlan(**memories[0].value)

    system_message += f"""
    This is the overall description of the coaching plan, and a summary of the previous sessions: 
    {coaching_plan.model_dump_json(indent=2)}
    Note in particular the homework left for this session on the last of the past sessions.
    """
    
    # Get turn
    turn_count = state.get("turn_count", 0)
    system_message += f"\nYou are now at this turn number in the conversation: {turn_count}"
    
    # Get summary if it exists
    summary = state.get("summary", "No summary yet")
    system_message += f"\n Summary of conversation earlier:\n{summary}"

    system_message += "\n\nLast messages:\n"
    
    messages = [SystemMessage(system_message)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": response, "turn_count": turn_count+1}


In [16]:
# Define a new graph
workflow_session = StateGraph(State)
workflow_session.add_node("conversation", call_model_session)
workflow_session.add_node(summarize_conversation)
workflow_session.add_node(finalize_session)

# Set the entrypoint as conversation
workflow_session.add_edge(START, "conversation")
workflow_session.add_conditional_edges("conversation", check_if_finished)
workflow_session.add_edge("summarize_conversation", END)
workflow_session.add_edge("finalize_session", END)

# Compile
graph = workflow_session.compile(checkpointer=checkpoint_memory, store=longterm_memory)
# display(Image(graph.get_graph().draw_mermaid_png())) --> the server isn't working 
ascii_diagram = graph.get_graph().draw_ascii()
print(ascii_diagram)

                                    +-----------+                            
                                    | __start__ |                            
                                    +-----------+                            
                                           *                                 
                                           *                                 
                                           *                                 
                                   +--------------+                          
                                   | conversation |                          
                              .....+--------------+......                    
                         .....             .             ......              
                   ......                   .                  .....         
                ...                         .                       ......   
+------------------+           +------------------------+       

In [17]:
# Create a thread
import uuid

thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id, "user_id": "Pablo"}}

In [21]:
for i in range(10):
    input_message = HumanMessage(content=input())
    output = graph.invoke({"messages": [input_message]}, config) 
    for m in output['messages']:
        m.pretty_print()
    if 'summary' in output.keys():
        print('\n----------\nSummary:\n',output['summary'])
    if checkpoint_memory.get(config)['channel_values'].get("finished")==True:
        print('CONVERSATION FINISHED')
        break

 ok


I'm asking to summarize
================================== Ai Message ==================================

I completely understand! Time can be a challenge. How about we focus on reviewing your progress with your Dutch studies and LinkedIn connections today? We can set a simple plan for the upcoming week that fits your schedule. Does that sound good?
================================ Human Message =================================

ok
================================== Ai Message ==================================

Great! Let’s start by reviewing your progress. How have your Dutch studies been going since our last session? Have you been able to meet any of the milestones you set?

----------
Summary:
 Pablo is making steady progress in his endeavors, including learning to play the guitar, which he finds fun. He has enrolled in a weekly Dutch course and started reading news articles in Dutch as part of his goal to achieve C1 proficiency. He is focused on reviewing his homework and discuss

 started the course


I'm asking to summarize
================================== Ai Message ==================================

Great! Let’s start by reviewing your progress. How have your Dutch studies been going since our last session? Have you been able to meet any of the milestones you set?
================================ Human Message =================================

started the course
================================== Ai Message ==================================

That's a fantastic step forward! Starting the course is a significant milestone. How are you finding the course so far? Are there any particular challenges or highlights you'd like to share?

----------
Summary:
 Pablo is making steady progress in his endeavors, including learning to play the guitar, which he finds fun. He has enrolled in a weekly Dutch course and started reading news articles in Dutch as part of his goal to achieve C1 proficiency. He is focused on reviewing his homework and discussing his progress towards his Dutch and 

 sorry, I need to go, see you next time


================================== Ai Message ==================================

Great! Let’s start by reviewing your progress. How have your Dutch studies been going since our last session? Have you been able to meet any of the milestones you set?
================================ Human Message =================================

started the course
================================== Ai Message ==================================

That's a fantastic step forward! Starting the course is a significant milestone. How are you finding the course so far? Are there any particular challenges or highlights you'd like to share?
================================ Human Message =================================

sorry, I need to go, see you next time
================================== Ai Message ==================================

No problem at all, Pablo! Thank you for the update. I look forward to hearing more about your progress next time. Take care!

----------
Summary:
 Pablo is making steady pro

In [22]:
namespace = ("session_plan", 'Pablo')
memories = longterm_memory.search(namespace)

coaching_plan = CoachingPlan(**memories[0].value)
print(coaching_plan.model_dump_json(indent=2))

{
  "plan_desciption": "This coaching plan aims to help Pablo enhance his professional profile and language skills, specifically targeting a C1 level in Dutch and improving his LinkedIn networking efforts.",
  "plan_summary": "Pablo is currently at a B1 level in Dutch and aims to reach C1 by following a course and joining a weekly chat club. He has started a weekly Dutch course and is reading news articles in Dutch as part of his goal to achieve C1 proficiency. He also wants to actively network on LinkedIn by connecting with 1-2 new people daily and posting about generative AI. The coaching program includes onboarding, goal setting, weekly sessions for progress tracking, an always-available chat for questions or exercises, and a closing session to review progress and define next steps. Pablo has been consistently connecting with 1-2 new people daily on LinkedIn and has noticed positive interactions from those connections, including work recommendations and valuable learnings.",
  "past